<h1><p style="text-align:center">Infrastructure Damage Prediction and Resource Optimization</p></h1>

# Business Problem

After a major earthquake, governments, NGOs, and environmental agencies allocate rescue teams, priortize inspections, distribute medical supplies, and allocate reconstruction funds but the downside of this is that inspecting every building immediately is impossible and resources that is been allocated by the concerned organizations is not sufficient putting into consideration that delays cost lives and also money.

As a data scientist i was tasked to with the below objectives.

# Business Objectives

Develop a predictive model that
1. Identifies buildings at high risk of severe damage
2. Prioritizes emergency response
3. Optimizes allocation of relief resources